In [ ]:
%%capture
# !pip3 install transformers==4.9.2
!pip3 install transformers
!pip3 install datasets
!pip3 install nltk

In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-2ff8084b-28be-3f16-a44b-a0f532be9906)


# Use updated code compatible for phoBERT
Update from https://github.com/luyug/Condenser

In [ ]:
%cd /content
!git clone https://github.com/legal-qa-research/zaloai-3pu-solution.git

/content
Cloning into 'zaloai-3pu-solution'...
remote: Enumerating objects: 2113, done.
remote: Counting objects: 100% (2113/2113), done.
remote: Compressing objects: 100% (1472/1472), done.
remote: Total 2113 (delta 601), reused 2102 (delta 590), pack-reused 0
Receiving objects: 100% (2113/2113), 10.47 MiB | 16.00 MiB/s, done.
Resolving deltas: 100% (601/601), done.


In [ ]:
!git add .
!git reset --hard
!git pull origin master

HEAD is now at d725155  update distil bert loss
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 3 (delta 2), reused 3 (delta 2), pack-reused 0
Unpacking objects: 100% (3/3), done.
From https://github.com/legal-qa-research/zaloai-3pu-solution
 * branch            master     -> FETCH_HEAD
   d725155..f165963  master     -> origin/master
Updating d725155..f165963
Fast-forward
 modeling.py | 4 ++--
 1 file changed, 2 insertions(+), 2 deletions(-)


In [ ]:
import os
os.environ['PYTHONPATH'] += ":/content/zaloai-3pu-solution"

## Data Preprocess

In [ ]:
ROOT_FOLDER = '/content/zaloai-3pu-solution'
CONDENSER_ROOT_FOLDER = '/content/zaloai-3pu-solution'
# DATA_FILE='/content/drive/Shareddrives/UnlimitedSharedDrive/legal_finetuned_data/small_bert_corpus.txt'
DATA_FILE='/content/drive/Shareddrives/UnlimitedSharedDrive/legal_finetuned_data/bert_corpus_path_v1.txt'
SAVE_CONDENSER='/content/drive/Shareddrives/UnlimitedSharedDrive/legal-qa-condenser-pretrained/segmented_train_data'
SAVE_MODEL_PATH = '/content/drive/Shareddrives/UnlimitedSharedDrive/legal-qa-condenser-pretrained/saved_phobert_ep2'

MODEL_NAME_OR_PATH = '/content/drive/Shareddrives/UnlimitedSharedDrive/legal-qa-condenser-pretrained/saved_phobert_ep1/checkpoint-32000'
TOKENIZER_NAME = '/content/drive/Shareddrives/UnlimitedSharedDrive/legal-qa-condenser-pretrained/saved_phobert_ep1/checkpoint-32000'

MAX_LENGTH=256
SAVE_STEP=1000
# MODEL_TYPE='bert'
BATCH_SIZE=16

In [ ]:
%cd $CONDENSER_ROOT_FOLDER
%run helper/create_train.py --tokenizer_name $TOKENIZER_NAME --file $DATA_FILE --save_to $SAVE_CONDENSER --max_len $MAX_LENGTH 

/content/zaloai-3pu-solution


Downloading:   0%|          | 0.00/557 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/874k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
  0%|          | 0/589489 [00:00<?, ?it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (262 > 256). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (265 > 256). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (257 > 256). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (257 > 256). Running this sequence through the model will result in indexing errors
100%|██████████| 589489/589489 [05:57<00:00, 1648.59it/s]


## Run training Condenser

In [ ]:
%cd $ROOT_FOLDER
# --train_dir $SAVE_CONDENSER
# --train_file $TRAIN_FILE
# --validation_file $DEV_FILE
%run run_pre_training.py --model_type=roberta --train_dir $SAVE_CONDENSER  --tokenizer_name $TOKENIZER_NAME --output_dir $SAVE_MODEL_PATH --model_name_or_path $MODEL_NAME_OR_PATH --max_seq_length $MAX_LENGTH --do_train --save_steps $SAVE_STEP --per_device_train_batch_size $BATCH_SIZE --gradient_accumulation_steps 4 --fp16 --warmup_ratio 0.1 --learning_rate 5e-5 --num_train_epochs 1 --overwrite_output_dir --dataloader_num_workers 4 --n_head_layers 2 --skip_from 6  --weight_decay 0.01  --late_mlm
# !python run_pre_training.py --validation_file $DEV_FILE --train_dir $SAVE_CONDENSER --tokenizer_name $TOKENIZER_NAME --output_dir $SAVE_MODEL_PATH --model_name_or_path $MODEL_NAME_OR_PATH --max_seq_length $MAX_LENGTH --do_train --save_steps 2000 --per_device_train_batch_size 32 --gradient_accumulation_steps 4 --fp16 --warmup_ratio 0.1 --learning_rate 5e-5 --num_train_epochs 8 --overwrite_output_dir --dataloader_num_workers 32 --n_head_layers 2 --skip_from 6  --weight_decay 0.01  --late_mlm

Step,Training Loss
500,2.647300
1000,2.637000
1500,2.667100
2000,2.676700
2500,2.634000
3000,2.620200
3500,2.580700
4000,2.571100
4500,2.582600
5000,2.541900


[INFO|trainer.py:1530] 2022-05-01 00:12:52,425 >> 

Training completed. Do not forget to share your model on huggingface.co/models =)


05/01/2022 00:12:52 - INFO - trainer -   Saving model checkpoint to /content/drive/Shareddrives/UnlimitedSharedDrive/legal-qa-condenser-pretrained/saved_phobert_ep2
[INFO|configuration_utils.py:441] 2022-05-01 00:12:52,441 >> Configuration saved in /content/drive/Shareddrives/UnlimitedSharedDrive/legal-qa-condenser-pretrained/saved_phobert_ep2/config.json
[INFO|modeling_utils.py:1378] 2022-05-01 00:12:54,123 >> Model weights saved in /content/drive/Shareddrives/UnlimitedSharedDrive/legal-qa-condenser-pretrained/saved_phobert_ep2/pytorch_model.bin
/content/zaloai-3pu-solution/modeling.py:126: UserWarning: omiting 205 transformer weights
  warnings.warn(f'omiting {len(hf_weight_keys)} transformer weights')
[INFO|tokenization_utils_base.py:2086] 2022-05-01 00:12:54,315 >> tokenizer config file saved in /content/drive/Shareddrives/UnlimitedSharedDrive/lega

In [ ]:
import transformers
transformers.__version__

'4.9.2'

GPU 0: Tesla T4 (UUID: GPU-13c2bd52-6767-a35e-a07a-17e8603fb85d)
